In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
)

template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

chain.predict(question="My name is Nico")



> Entering new LLMChain chain...
Prompt after formatting:
My name is Nico

> Finished chain.


'Nice to meet you, Nico! How can I assist you today?'

In [2]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:
I live in Seoul

> Finished chain.


"That's great! Seoul is the capital and largest city of South Korea. It is known for its vibrant culture, modern architecture, and delicious cuisine. There are many attractions to explore in Seoul, such as Gyeongbokgung Palace, N Seoul Tower, Myeongdong shopping district, and the Han River. The city also offers a bustling nightlife scene and a wide range of entertainment options. Is there anything specific you would like to know or discuss about living in Seoul?"

In [3]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
What is my name?

> Finished chain.


"I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation."

In [4]:
memory.load_memory_variables({"inputs": "what does nicolas like"})

{'history': "System: The human introduces themselves as Nico. The AI greets Nico and asks how it can assist them. Nico mentions that they live in Seoul, and the AI provides information about the city, including its vibrant culture, modern architecture, delicious cuisine, and various attractions. The AI also mentions the bustling nightlife scene and entertainment options in Seoul. The AI asks if there is anything specific Nico would like to know or discuss about living in Seoul.\nHuman: What is my name?\nAI: I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation."}